In [ ]:
# Make One Big String so the model can learn its cohesive structre for next token, etc.
import pandas as pd

# Load your dataset
df = pd.read_csv("Songs/OUTPUTS/AllSongs_Cleaned.csv", sep='\t', encoding='utf-8')

# Optional: sort if necessary (you want the original order of words)
#df = df.sort_values(by=['line_number', 'token_number'])

# Concatenate into a single string
text = ' '.join(df['chunk_str'].astype(str).tolist())

# Let's see what it looks like
print(text[:300])

nan 'Duke  Duke  Duke  Duke of Earl' 'Duke  Duke  Duke of Earl' 'Duke  Duke  Duke of Earl' 'Duke  Duke  Duke of Earl' 'Duke  Duke  Duke of Earl' 'Duke  Duke  Duke of Earl' 'Duke  Duke  Duke of Earl' 'Duke  Duke  Duke of Earl' 'As I walk through this world' 'Nothing can stop the Duke of Earl' 'And-a 


In [12]:
from transformers import GPT2Tokenizer, GPT2LMHeadModel
import torch
from torch.utils.data import Dataset

# Load pre-trained tokenizer
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')

# Optional: Add special tokens if needed
tokenizer.pad_token = tokenizer.eos_token  # GPT2 doesn't have a pad token, just reuse eos

# Tokenize your text
tokens = tokenizer(text, return_tensors='pt', truncation=True)

# Check out the size
print(tokens['input_ids'].shape)

c:\Users\grozz\miniconda3\envs\gemcordelli\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Users\grozz\miniconda3\envs\gemcordelli\Lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\grozz\.cache\huggingface\hub\models--gpt2. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate develope

torch.Size([1, 1024])


In [13]:
class LyricsDataset(Dataset):
    def __init__(self, tokens, block_size=128):
        self.input_ids = tokens['input_ids'][0]
        self.block_size = block_size

    def __len__(self):
        return len(self.input_ids) - self.block_size

    def __getitem__(self, idx):
        x = self.input_ids[idx:idx + self.block_size]
        y = self.input_ids[idx + 1:idx + 1 + self.block_size]
        return x, y

dataset = LyricsDataset(tokens, block_size=64)

In [17]:
from transformers import GPT2LMHeadModel
from torch.optim import AdamW

model = GPT2LMHeadModel.from_pretrained('gpt2')
model.resize_token_embeddings(len(tokenizer))  # adjust vocab size if you changed it
model.train()

optimizer = AdamW(model.parameters(), lr=5e-5)

# Training loop (tiny version)
for epoch in range(3):
    for i in range(0, len(dataset), 32):  # batch of 32
        batch = [dataset[j] for j in range(i, min(i + 32, len(dataset)))]
        x_batch = torch.stack([item[0] for item in batch])
        y_batch = torch.stack([item[1] for item in batch])

        outputs = model(input_ids=x_batch, labels=y_batch)
        loss = outputs.loss
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        if i % 256 == 0:
            print(f"Epoch {epoch} | Step {i} | Loss: {loss.item():.4f}")

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`
`loss_type=None` was set in the config but it is unrecognised.Using the default loss: `ForCausalLMLoss`.


Epoch 0 | Step 0 | Loss: 7.1051
Epoch 0 | Step 256 | Loss: 5.7210
Epoch 0 | Step 512 | Loss: 5.7310
Epoch 0 | Step 768 | Loss: 6.0344
Epoch 1 | Step 0 | Loss: 3.8072
Epoch 1 | Step 256 | Loss: 4.8660
Epoch 1 | Step 512 | Loss: 4.8592
Epoch 1 | Step 768 | Loss: 5.1227
Epoch 2 | Step 0 | Loss: 3.4993
Epoch 2 | Step 256 | Loss: 4.0413
Epoch 2 | Step 512 | Loss: 3.9977
Epoch 2 | Step 768 | Loss: 4.1741


In [23]:
model.eval()
prompt = "love is"
inputs = tokenizer(prompt, return_tensors='pt')
outputs = model.generate(inputs['input_ids'], max_length=50, do_sample=True, top_k=40)
print(tokenizer.decode(outputs[0]))

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


love is  You your  You wanna it' Cause butt  round   round  to waist - round round    To your round' 'm of -ake I it  And't me't  I of


{'input_ids': tensor([[12647,   705,    35,  ...,   705,  6104,  2330]]), 'attention_mask': tensor([[1, 1, 1,  ..., 1, 1, 1]])}